In [1]:
# !git clone https://github.com/open-mmlab/mmdetection.git

In [2]:
# !git clone https://github.com/SwinTransformer/Swin-Transformer-Object-Detection.git

In [2]:
# from IPython.display import clear_output

# %cd /app/_data/mmdetection
# !pip install -r requirements/build.txt
# !pip install -v -e .  # or "python setup.py develop"
# clear_output()

In [3]:
# from IPython.display import clear_output
# !pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.1/index.html
# !pip install openmim
# !mim install mmdet
# clear_output()

In [1]:
!pip install mmcv-full==1.4.0 -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.1/index.html

Looking in links: https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.1/index.html
     |████████████████████████████████| 403 kB 2.0 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for mmcv-full: filename=mmcv_full-1.4.0-cp38-cp38-linux_x86_64.whl size=55221063 sha256=cca1564f7f842713b7b147a05985716bdf253986f08f493eac2d719d69001912
  Stored in directory: /root/.cache/pip/wheels/40/af/51/3a4996d6bf2544246b66190f1fdcaa48663efe6535844ed702
Successfully built mmcv-full
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [2]:
%cd /app/_data/Swin-Transformer-Object-Detection/apex
# !pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
!python setup.py install

/app/_data/Swin-Transformer-Object-Detection/apex


torch.__version__  = 1.9.1+cu111


setup.py:109: UserWarning: Option --pyprof not specified. Not installing PyProf dependencies!
  warnings.warn("Option --pyprof not specified. Not installing PyProf dependencies!")
running install
running bdist_egg
running egg_info
writing apex.egg-info/PKG-INFO
writing dependency_links to apex.egg-info/dependency_links.txt
writing top-level names to apex.egg-info/top_level.txt
reading manifest file 'apex.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'apex.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py

creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/apex
creating build/bdist.linux-x86_64/egg/apex/parallel
copying build/lib/apex/parallel/sync_batchnorm_kernel.py -> build/bdist.linux-x86_64/egg/apex/parallel
copying build/lib/apex/parallel/LARC.py -> build/bdist.linux-x86_64/egg/a

In [6]:
%cd /app/_data/Swin-Transformer-Object-Detection
!pip install -r requirements/build.txt
!pip install -v -e .  

/app/_data/Swin-Transformer-Object-Detection
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
Using pip 21.3.1 from /usr/local/lib/python3.8/dist-packages/pip (python 3.8)
Obtaining file:///app/_data/Swin-Transformer-Object-Detection
  Preparing metadata (setup.py) ...   Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-w3z3jxl_/mmdet.egg-info
  writing /tmp/pip-pip-egg-info-w3z3jxl_/mmdet.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-w3z3jxl_/mmdet.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-pip-egg-info-w3z3jxl_/mmdet.egg-info/requires.txt
  writing top-level names to /tmp/pip-pip-egg-info-w3z3jxl_/mmdet.egg-info/top_level.txt
  writing manifest file '/tmp/pip-pip-egg-info-w3z3jxl_/mmdet.egg-info/SOURCES.txt'
  reading manifest file '/tmp/pip-pip-egg-info-w3z3jxl_/mmdet.egg-info/SOURCES.txt'
  adding license file 'LICENSE'
  writing manifest file '/tmp/

In [7]:
import mmcv
import mmdet
from mmcv import Config
from mmdet.apis import (
    inference_detector,
    init_detector,
    set_random_seed,
    train_detector,
)
from mmdet.datasets import build_dataset
from mmdet.models import build_detector

In [7]:
# !git clone https://github.com/NVIDIA/apex


In [8]:
# %cd apex
# !pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

In [58]:
SEED = 37
set_random_seed(SEED, deterministic=False)

In [59]:
%cd /app/_data/Swin-Transformer-Object-Detection
import argparse
import ast
import glob
import json
import os
import re
import shutil

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from IPython.core.interactiveshell import InteractiveShell
from PIL import Image
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm

InteractiveShell.ast_node_interactivity = "all"
import seaborn as sns
import torch
import torchvision

/app/_data/Swin-Transformer-Object-Detection


In [60]:
torch.__version__

'1.9.1+cu111'

In [61]:
def plot_bboxes(img, bboxes, bbox_format="coco"):
    try:
        img_h, img_w = img.shape[:2]
    except:
        img_w, img_h = img.size
    fig, ax = plt.subplots(1, 1, figsize=(20, 12))
    ax.imshow(img)
    if bbox_format == "coco":
        for i in range(len(bboxes)):
            if len(bboxes[i].split()) == 5:
                bbox = bboxes[i].split()[1:]
            else:
                bbox = bboxes[i].split()
            x_center, y_center, width, height = list(map(float, bbox))
            x_min = int((x_center - width / 2) * img_w)
            y_min = int((y_center - height / 2) * img_h)
            width = int(width * img_w)
            height = int(height * img_h)
            rect = plt.Rectangle(
                [x_min, y_min], width, height, ec="r", fc="none", lw=2.0
            )
            ax.add_patch(rect)
    elif bbox_format == "from_df":
        for i in range(len(bboxes)):
            x_min, y_min, width, height = (
                bboxes[i]["x"],
                bboxes[i]["y"],
                bboxes[i]["width"],
                bboxes[i]["height"],
            )
            rect = plt.Rectangle(
                [x_min, y_min], width, height, ec="r", fc="none", lw=2.0
            )
            ax.add_patch(rect)
    plt.show();

In [62]:
TRAIN_DF_PART = "/app/_data/tensorflow-great-barrier-reef/train.csv"
IMAGE_FOLDER = "images"
LABEL_FOLDER = "labels"

In [63]:
with open("/app/_data/sequences.json", "r") as f:
    seq_dict = json.load(f)

In [64]:
df = pd.read_csv(TRAIN_DF_PART)
df["img_path"] = (
    "/app/_data/tensorflow-great-barrier-reef/train_images/video_"
    + df.video_id.astype("str")
    + "/"
    + df.video_frame.astype("str")
    + ".jpg"
)
df["annotations"] = df["annotations"].apply(lambda x: ast.literal_eval(x))
df["len_annotation"] = df["annotations"].str.len()
df["image_id"] = df["image_id"].str.replace("-", "_", regex=True)
df["new_img_path"] = f"/app/_data/{IMAGE_FOLDER}/" + df["image_id"] + ".jpg"
df["label"] = df["len_annotation"].apply(lambda x: 0 if x == 0 else 1)
df["no_label"] = df["len_annotation"].apply(lambda x: True if x == 0 else False)
R = df[df["len_annotation"] == 0].shape[0] / df[df["len_annotation"] != 0].shape[0]
df["label_change"] = df["label"] & df["no_label"].shift(1) & df["no_label"].shift(
    2
) | df["no_label"] & df["label"].shift(1) & df["label"].shift(2)
df["sequense_change"] = df["sequence"] != df["sequence"].shift(1)
df["start_subseq"] = df["sequense_change"] | df["label_change"]
df.loc[df.index[-1], "start_subseq"] = True
df["start_subseq"].sum()
start_idx = 0
for subsequence_id, end_idx in enumerate(df[df["start_subseq"]].index):
    df.loc[start_idx:end_idx, "subsequence_id"] = subsequence_id
    start_idx = end_idx

df["subsequence_id"] = df["subsequence_id"].astype(int)
df["subsequence_id"].nunique()

138

137

# train_test_split

## random

In [65]:
# l = 0
# ll = 0
# for i in tqdm(range(2000)):
#     n_val = np.random.randint(15, 25, 1)[0]
#     val_seq = sample(df.subsequence_id.unique().tolist(), n_val)
#     train_seq = list(set(df.subsequence_id.tolist()) - set(val_seq))
#     l = (
#         df.query("subsequence_id in @train_seq")["label"].sum()
#         / df.query("subsequence_id in @val_seq")["label"].sum()
#     )
#     ll = (
#         df.query("subsequence_id in @train_seq")["len_annotation"].sum()
#         / df.query("subsequence_id in @val_seq")["len_annotation"].sum()
#     )
#     r = (
#         df.query("subsequence_id in @val_seq and len_annotation == 0").shape[0]
#         / df.query("subsequence_id in @val_seq and len_annotation != 0").shape[0]
#     )
#     if 8 <= l <= 10 and 8 <= ll <= 10 and 3.5 <= r <= 4.3:
#         break

# df.query("subsequence_id in @train_seq")[["len_annotation", "label"]].sum() / df.query(
#     "subsequence_id in @val_seq"
# )[["len_annotation", "label"]].sum()

# df.query("subsequence_id in @val_seq and len_annotation == 0").shape[0] / df.query(
#     "subsequence_id in @val_seq and len_annotation != 0"
# ).shape[0]

# sorted(val_seq)

In [66]:
# KFOLD = "rand"
# train_idx = df.query("subsequence_id in @train_seq and len_annotation>0").index.tolist()
# val_idx = df.query("subsequence_id in @val_seq").index.tolist()
# train = df.loc[train_idx]
# val = df.loc[val_idx]
# train[["len_annotation", "label"]].sum()
# val[["len_annotation", "label"]].sum()
# train[["len_annotation", "label"]].sum() / val[["len_annotation", "label"]].sum()

## video_id

In [67]:
VIDEO_ID = 2
train  = pd.concat(
    [
        df.query("video_id!=@VIDEO_ID and len_annotation!=0"),
        df.query("video_id!=@VIDEO_ID and len_annotation==0").sample(
            int(
                df.query("video_id!=@VIDEO_ID and len_annotation!=0").shape[0]
                * 0.07
            )
        ),
    ]
).sample(frac = 1)
val = df.query("video_id==@VIDEO_ID").sample(frac = 1)

## StratifiedKFold on subsequence_id

In [68]:
# df_split = (
#     df.groupby("subsequence_id")
#     .agg({"label": "max", "len_annotation": "sum", "video_frame": "count"})
#     .astype(int)
#     .reset_index()
# )
# n_splits = 10
# y=df_split["label"]
# skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)

# for fold_id, (train_idx, val_idx) in enumerate(
#     skf.split(df_split["subsequence_id"], y=y)
# ):
#     subseq_val_idx = df_split["subsequence_id"].iloc[val_idx]
#     df.loc[df["subsequence_id"].isin(subseq_val_idx), "fold"] = fold_id

# df["fold"] = df["fold"].astype(int)
# for fold in range(10):
#     print(f"\nFold {fold}")
#     df.query("fold != @fold")[["len_annotation", "label"]].sum() / df.query(
#         "fold == @fold"
#     )[["len_annotation", "label"]].sum()

In [69]:
# KFOLD = 3
# train = pd.concat(
#     [
#         df.query("fold != @KFOLD and len_annotation!=0"),
#         df.query("fold != @KFOLD and len_annotation==0").sample(
#             int(df.query("fold != @KFOLD and len_annotation!=0").shape[0] * 0.07)
#         ),
#     ]
# ).sample(frac=1)
# val = df.query("fold == @KFOLD").sample(frac=1)
# train[["len_annotation", "label"]].sum() / val[["len_annotation", "label"]].sum()

# train[["len_annotation", "label"]].sum()

# val[["len_annotation", "label"]].sum()

# train_ids = train.index.tolist()
# val_ids = val.index.tolist()

# len(train_ids), len(val_ids)

# train_img_path = df.loc[train_ids, "new_img_path"].tolist()
# val_img_path = df.loc[val_ids, "new_img_path"].tolist()
# np.savetxt(
#     f"/app/_data/train_{SEED}_{KFOLD}.txt",
#     train_img_path,
#     fmt="%s",
# )
# np.savetxt(f"/app/_data/val_{SEED}_{KFOLD}.txt", val_img_path, fmt="%s")

In [70]:
%%writefile /app/_data/for_mmdet/labels.txt 
starfish

Overwriting /app/_data/for_mmdet/labels.txt


In [71]:
def df_to_coco(df):
    images = []
    annotations = []
    img_id = 0
    annotation_id = 0
    categories = [
        {
            "id": 0,
            "name": "starfish",
        },
    ]
    for ix in tqdm(df.index.tolist()):
        img = Image.open(df.loc[ix, "img_path"])
        w, h = img.size
        bboxes = df.loc[ix, "annotations"]
        img_name = f'{df.loc[ix, "image_id"]}.jpg'
        img_dict = {
            "id": img_id,
            "file_name": img_name,
            "height": h,
            "width": w,
        }
        images.append(img_dict)
        if len(bboxes) != 0:
            for i in range(len(bboxes)):
                bbox = bboxes[i]
                bbox = [bbox["x"], bbox["y"], bbox["width"], bbox["height"]]
                annotations_dict = {
                    "id": annotation_id,
                    "image_id": img_id,
                    "category_id": 0,
                    "bbox": bbox,
                    "area": bbox[2] * bbox[3],
                    "segmentation": [],
                    "iscrowd": 0,
                }
                annotations.append(annotations_dict)
                annotation_id += 1
        img_id += 1
    coco_dict = {"categories": categories, "images": images, "annotations": annotations}
    return coco_dict

In [72]:
train_dict = df_to_coco(train)
with open('/app/_data/for_mmdet/coco_train.json', 'w') as f:
    json.dump(train_dict, f, ensure_ascii=True, indent=4)
val_dict = df_to_coco(val)
with open('/app/_data/for_mmdet/coco_val.json', 'w') as f:
    json.dump(val_dict, f, ensure_ascii=True, indent=4)

100% 4538/4538 [00:00<00:00, 5622.72it/s]
100% 8561/8561 [00:01<00:00, 6258.78it/s]


# Model Config

In [73]:
cfg = Config.fromfile(
    "/app/_data/Swin-Transformer-Object-Detection/configs/swin/cascade_mask_rcnn_swin_tiny_patch4_window7_mstrain_480-800_giou_4conv1f_adamw_3x_coco.py"
)

In [74]:
# cfg

In [75]:
model_name = "cascade_mask_rcnn_swin_tyny_patch_1"
# Folder to store model logs and weight files
work_dir = f"/app/_data/for_mmdet/{model_name}"
cfg.work_dir = work_dir
if not os.path.exists(work_dir):
    os.makedirs(work_dir)

In [76]:
# cfg.model.roi_head

In [77]:
for head in cfg.model.roi_head.bbox_head:
    head.num_classes = 1

In [78]:
cfg.model.backbone

{'type': 'SwinTransformer',
 'embed_dim': 96,
 'depths': [2, 2, 6, 2],
 'num_heads': [3, 6, 12, 24],
 'window_size': 7,
 'mlp_ratio': 4.0,
 'qkv_bias': True,
 'qk_scale': None,
 'drop_rate': 0.0,
 'attn_drop_rate': 0.0,
 'drop_path_rate': 0.2,
 'ape': False,
 'patch_norm': True,
 'out_indices': (0, 1, 2, 3),
 'use_checkpoint': False}

In [79]:
cfg.optimizer

{'type': 'AdamW',
 'lr': 0.0001,
 'betas': (0.9, 0.999),
 'weight_decay': 0.05,
 'paramwise_cfg': {'custom_keys': {'absolute_pos_embed': {'decay_mult': 0.0},
   'relative_position_bias_table': {'decay_mult': 0.0},
   'norm': {'decay_mult': 0.0}}}}

In [80]:
cfg.runner.max_epochs = 30

In [81]:
cfg.lr_config

{'policy': 'step',
 'warmup': 'linear',
 'warmup_iters': 500,
 'warmup_ratio': 0.001,
 'step': [27, 33]}

In [82]:
# cfg.lr_config = dict(
#     policy='CosineAnnealing', # The policy of scheduler, also support CosineAnnealing, Cyclic, etc. Refer to details of supported LrUpdater from https://github.com/open-mmlab/mmcv/blob/master/mmcv/runner/hooks/lr_updater.py#L9.
#     by_epoch=False,
#     warmup='linear', # The warmup policy, also support `exp` and `constant`.
#     warmup_iters=500, # The number of iterations for warmup
#     warmup_ratio=0.001, # The ratio of the starting learning rate used for warmup
#     min_lr=1e-07)

In [83]:
cfg.log_config.interval = 200

In [84]:
cfg.data.samples_per_gpu=8
cfg.data.workers_per_gpu=4

In [85]:
cfg.dataset_type

'CocoDataset'

In [86]:
cfg.gpu_ids = [0]
cfg.classes = '/app/_data/for_mmdet/labels.txt'

In [87]:
cfg.data.train.ann_file =  "/app/_data/for_mmdet/coco_train.json"
cfg.data.train.img_prefix =  "/app/_data/images/"
cfg.data.train.classes = cfg.classes

In [88]:
cfg.data.val.ann_file =  "/app/_data/for_mmdet/coco_val.json"
cfg.data.val.img_prefix =  "/app/_data/images/"
cfg.data.val.classes = cfg.classes

In [89]:
cfg.data.test = cfg.data.val

In [90]:
cfg.seed = SEED

In [91]:
cfg.evaluation.metric # = ['bbox']

['bbox', 'segm']

In [92]:
import wandb
wandb.login()

True

In [93]:
cfg.log_config.hooks

[{'type': 'TextLoggerHook'}]

In [94]:
cfg.log_config.hooks = [dict(type='TextLoggerHook'),
                        dict(type='WandbLoggerHook',
                             init_kwargs=dict(project='cascade_101',
                                              name=f'exp-{model_name}',
                                              entity='tatanko'))]

In [95]:
cfg.data.train.pipeline

[{'type': 'LoadImageFromFile'},
 {'type': 'LoadAnnotations', 'with_bbox': True, 'with_mask': True},
 {'type': 'RandomFlip', 'flip_ratio': 0.5},
 {'type': 'AutoAugment',
  'policies': [[{'type': 'Resize',
     'img_scale': [(480, 1333),
      (512, 1333),
      (544, 1333),
      (576, 1333),
      (608, 1333),
      (640, 1333),
      (672, 1333),
      (704, 1333),
      (736, 1333),
      (768, 1333),
      (800, 1333)],
     'multiscale_mode': 'value',
     'keep_ratio': True}],
   [{'type': 'Resize',
     'img_scale': [(400, 1333), (500, 1333), (600, 1333)],
     'multiscale_mode': 'value',
     'keep_ratio': True},
    {'type': 'RandomCrop',
     'crop_type': 'absolute_range',
     'crop_size': (384, 600),
     'allow_negative_crop': True},
    {'type': 'Resize',
     'img_scale': [(480, 1333),
      (512, 1333),
      (544, 1333),
      (576, 1333),
      (608, 1333),
      (640, 1333),
      (672, 1333),
      (704, 1333),
      (736, 1333),
      (768, 1333),
      (800, 1333)]

In [96]:
cfg.data.train.pipeline = cfg.train_pipeline

In [38]:
albu_train_transforms = [
    dict(type='ShiftScaleRotate', shift_limit=0.0625,
         scale_limit=0.15, rotate_limit=15, p=0.05),
    dict(type='RandomBrightnessContrast', brightness_limit=0.2,
         contrast_limit=0.2, p=0.3),
    dict(type='IAAAffine', shear=(-10.0, 10.0), p=0.4),
    dict(type="Blur", p=1.0, blur_limit=7),
    dict(type='CLAHE', p=0.5),
    dict(type='Equalize', mode='cv', p=0.4),
    dict(
        type="OneOf",
        transforms=[
            dict(type="GaussianBlur", p=1.0, blur_limit=7),
            dict(type="MedianBlur", p=1.0, blur_limit=7),
        ],
        p=0.4,
    ),
    
    dict(type='MixUp', p=0.2, lambd=0.5),
    dict(type='Flip', p=0.5),
    ]


cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=False),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Albu',
        transforms=albu_train_transforms,
        bbox_params=dict(
        type='BboxParams',
        format='pascal_voc',
        label_fields=['gt_labels'],
        min_visibility=0.5,
        filter_lost_elements=True),
        keymap=dict(img='image', gt_bboxes='bboxes'),
        update_pad_shape=False,
        skip_img_without_anno=True),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels', 'gt_masks'])
]
cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1333, 800),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(
                type='Normalize',
                mean=[123.675, 116.28, 103.53],
                std=[58.395, 57.12, 57.375],
                to_rgb=True),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img'])
        ])
]

In [97]:
model = build_detector(
    cfg.model, train_cfg=cfg.get("train_cfg"), test_cfg=cfg.get("test_cfg")
)
model.init_weights()

In [98]:
train_dataset = build_dataset(cfg.data.train)

loading annotations into memory...
Done (t=0.19s)
creating index...
index created!


In [99]:
train_dataset

In [100]:
train_detector(model, train_dataset, cfg, distributed=False, validate=True)

AttributeError: module 'apex' has no attribute 'amp'

In [107]:
%cd /app/_data/Swin-Transformer-Object-Detection
from apex import amp

/app/_data/Swin-Transformer-Object-Detection


ImportError: cannot import name 'amp' from 'apex' (unknown location)

In [101]:
import apex

In [16]:
%%writefile /app/_data/mmdetection/configs/swin/TFGBR_swin_base_faster_rcnn_fp16.py

_base_ = [
    "/app/_data/mmdetection/configs/_base_/models/faster_rcnn_r50_fpn.py",
    "/app/_data/mmdetection/configs/_base_/datasets/coco_detection.py",
    "/app/_data/mmdetection/configs/_base_/schedules/schedule_2x.py",
    "/app/_data/mmdetection/configs/_base_/default_runtime.py",
]
pretrained = "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_small_patch4_window7_224.pth"
model = dict(
    backbone=dict(
        _delete_=True,
        type="SwinTransformer",
        embed_dims=96,
        depths=[2, 2, 18, 2],
        num_heads=[3, 6, 12, 24],
        window_size=7,
        mlp_ratio=4,
        qkv_bias=True,
        qk_scale=None,
        drop_rate=0.0,
        attn_drop_rate=0.0,
        drop_path_rate=0.2,
        patch_norm=True,
        out_indices=(0, 1, 2, 3),
        with_cp=False,
        convert_weights=True,
        init_cfg=dict(type="Pretrained", checkpoint=pretrained),
    ),
    neck=dict(in_channels=[96, 192, 384, 768]),
    roi_head=dict(
        bbox_head=dict(
            reg_decoded_bbox=True, loss_bbox=dict(type="GIoULoss", loss_weight=10.0)
        )
    ),
)

optimizer = dict(
    _delete_=True,
    type="AdamW",
    lr=0.0005,
    betas=(0.9, 0.999),
    weight_decay=0.05,
    paramwise_cfg=dict(
        custom_keys={
            "absolute_pos_embed": dict(decay_mult=0.0),
            "relative_position_bias_table": dict(decay_mult=0.0),
            "norm": dict(decay_mult=0.0),
        }
    ),
)
lr_config = dict(warmup_iters=500, step=[8, 11])
runner = dict(max_epochs=50)

Overwriting /app/_data/mmdetection/configs/swin/TFGBR_swin_base_faster_rcnn_fp16.py


In [17]:
cfg = Config.fromfile(
    "/app/_data/mmdetection/configs/swin/TFGBR_swin_base_faster_rcnn_fp16.py"
)

In [18]:
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True
)
train_pipeline = [
    dict(type="LoadImageFromFile", to_float32=True),
    dict(type="LoadAnnotations", with_bbox=True),
    dict(
        type="AutoAugment",
        policies=[
            [
                dict(
                    type="Resize",
                    img_scale=[
                        (480, 1333),
                        (512, 1333),
                        (544, 1333),
                        (576, 1333),
                        (608, 1333),
                        (640, 1333),
                        (672, 1333),
                        (704, 1333),
                        (736, 1333),
                        (768, 1333),
                        (800, 1333),
                    ],
                    multiscale_mode="value",
                    keep_ratio=True,
                )
            ],
            [
                dict(
                    type="Resize",
                    img_scale=[(400, 1333), (500, 1333), (600, 1333)],
                    multiscale_mode="value",
                    keep_ratio=True,
                ),
                dict(
                    type="RandomCrop",
                    crop_type="absolute_range",
                    crop_size=(384, 600),
                    allow_negative_crop=True,
                ),
                dict(
                    type="Resize",
                    img_scale=[
                        (480, 1333),
                        (512, 1333),
                        (544, 1333),
                        (576, 1333),
                        (608, 1333),
                        (640, 1333),
                        (672, 1333),
                        (704, 1333),
                        (736, 1333),
                        (768, 1333),
                        (800, 1333),
                    ],
                    multiscale_mode="value",
                    override=True,
                    keep_ratio=True,
                ),
                dict(
                    type="PhotoMetricDistortion",
                    brightness_delta=32,
                    contrast_range=(0.5, 1.5),
                    saturation_range=(0.5, 1.5),
                    hue_delta=18,
                ),
                dict(
                    type="MinIoURandomCrop",
                    min_ious=(0.4, 0.5, 0.6, 0.7, 0.8, 0.9),
                    min_crop_size=0.3,
                ),
                dict(
                    type="CutOut",
                    n_holes=(5, 10),
                    cutout_shape=[
                        (4, 4),
                        (4, 8),
                        (8, 4),
                        (8, 8),
                        (16, 32),
                        (32, 16),
                        (32, 32),
                        (32, 48),
                        (48, 32),
                        (48, 48),
                    ],
                ),
            ],
        ],
    ),
    dict(type="RandomFlip", flip_ratio=0.5),
    dict(type="Normalize", **img_norm_cfg),
    dict(type="Pad", size_divisor=32),
    dict(type="DefaultFormatBundle"),
    dict(type="Collect", keys=["img", "gt_bboxes", "gt_labels"]),
]

test_pipeline = [
    dict(type="LoadImageFromFile"),
    dict(
        type="MultiScaleFlipAug",
        img_scale=(1333, 800),
        flip=False,
        transforms=[
            dict(type="Resize", keep_ratio=True),
            dict(type="RandomFlip"),
            dict(type="Normalize", **img_norm_cfg),
            dict(type="Pad", size_divisor=32),
            dict(type="DefaultFormatBundle"),
            dict(type="Collect", keys=["img"]),
        ],
    ),
]

In [19]:
work_dir = f"/app/_data/for_mmdet/frcnn"
if not os.path.exists(work_dir):
    os.makedirs(work_dir)

In [20]:
cfg.classes = "/app/_data/for_mmdet/labels.txt"
cfg.work_dir =work_dir
cfg.data_root = "/app/_data"

cfg.model.roi_head.bbox_head.num_classes = 1

cfg.data.test.type = "CocoDataset"
cfg.data.test.classes = "/app/_data/for_mmdet/labels.txt"
cfg.data.test.ann_file = "/app/_data/for_mmdet/coco_val.json"
cfg.data.test.img_prefix = "/app/_data/images/"

cfg.data.train.type = "CocoDataset"
cfg.data.train.ann_file = "/app/_data/for_mmdet/coco_train.json"
cfg.data.train.img_prefix = "/app/_data/images"
cfg.data.train.classes = "/app/_data/for_mmdet/labels.txt"

cfg.data.val.type = "CocoDataset"
cfg.data.val.ann_file = "/app/_data/for_mmdet/coco_val.json"
cfg.data.val.img_prefix = "/app/_data/images"
cfg.data.val.classes = "/app/_data/for_mmdet/labels.txt"

cfg.data.samples_per_gpu = 4
cfg.data.workers_per_gpu = 4

cfg.train_pipeline = train_pipeline
cfg.val_pipeline = test_pipeline
cfg.test_pipeline = test_pipeline

cfg.data.train.pipeline = cfg.train_pipeline
cfg.data.val.pipeline = cfg.val_pipeline
cfg.data.test.pipeline = cfg.test_pipeline

# cfg.optimizer.lr = 0.002 / 8
# cfg.lr_config = dict(
#     policy='CosineAnnealing',
#     by_epoch=False,
#     warmup='exp',
#     warmup_iters= 500,
#     warmup_ratio= 1/12,
#     min_lr=1e-05)

# cfg.runner.max_epochs = 12
cfg.evaluation.interval = 1
cfg.evaluation.save_best = "auto"

cfg.seed = SEED
cfg.gpu_ids = range(1)

cfg.fp16 = dict(loss_scale=dict(init_scale=512.0))

cfg.log_config = dict(
    interval=200,
    hooks=[dict(type="TextLoggerHook"), dict(type="TensorboardLoggerHook")],
)

meta = dict()
meta["config"] = cfg.pretty_text

In [21]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


In [22]:
datasets

[
 CocoDataset Train dataset with number of images 4242, and instance counts: 
 +--------------+-------+----------+-------+----------+-------+----------+-------+----------+-------+
 | category     | count | category | count | category | count | category | count | category | count |
 +--------------+-------+----------+-------+----------+-------+----------+-------+----------+-------+
 |              |       |          |       |          |       |          |       |          |       |
 | 0 [starfish] | 9449  |          |       |          |       |          |       |          |       |
 +--------------+-------+----------+-------+----------+-------+----------+-------+----------+-------+]

In [23]:
model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

In [24]:
model.init_weights()

2022-02-11 12:28:03,410 - mmdet - INFO - load checkpoint from http path: https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_small_patch4_window7_224.pth
Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_small_patch4_window7_224.pth" to /root/.cache/torch/hub/checkpoints/swin_small_patch4_window7_224.pth


  0%|          | 0.00/191M [00:00<?, ?B/s]

2022-02-11 12:28:07,997 - mmdet - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2022-02-11 12:28:08,017 - mmdet - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2022-02-11 12:28:08,023 - mmdet - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override': [{'name': 'shared_fcs'}, {'name': 'cls_fcs'}, {'name': 'reg_fcs'}]}]
2022-02-11 12:28:08,138 - mmcv - INFO - 
backbone.patch_embed.projection.weight - torch.Size([96, 3, 4, 4]): 
Initialized by user-defined `init_weights` in SwinTransformer  
 
2022-02-11 12:28:08,139 - mmcv - INFO - 
backbone.patch_embed.projection.bias - torch.Size([96]): 
Initialized by user-defined `init_weights` in SwinTransformer  
 
2022-02-11 12:28:08,140 - mmcv - INFO - 
backbone.patch_

In [25]:
model.CLASSES = datasets[0].CLASSES

In [26]:
mmcv.mkdir_or_exist(os.path.abspath(cfg.work_dir))

In [27]:
train_detector(model, datasets, cfg, distributed=False, validate=True, meta = meta)

2022-02-11 12:28:26,111 - mmdet - INFO - Start running, host: root@13ada95134ff, work_dir: /app/_data/for_mmdet/frcnn
2022-02-11 12:28:26,112 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(ABOVE_NORMAL) Fp16OptimizerHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) TensorboardLoggerHook              
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) TensorboardLoggerHook              
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook      

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


2022-02-11 12:30:22,442 - mmdet - INFO - Epoch [1][200/1061]	lr: 1.993e-04, eta: 8:26:16, time: 0.575, data_time: 0.024, memory: 16101, loss_rpn_cls: 0.1689, loss_rpn_bbox: 0.0187, loss_cls: 0.0984, acc: 96.9094, loss_bbox: 0.0504, loss: 0.3364
2022-02-11 12:32:16,127 - mmdet - INFO - Epoch [1][400/1061]	lr: 3.991e-04, eta: 8:21:34, time: 0.568, data_time: 0.010, memory: 18436, loss_rpn_cls: 0.0538, loss_rpn_bbox: 0.0181, loss_cls: 0.1003, acc: 97.2458, loss_bbox: 0.1013, loss: 0.2735
2022-02-11 12:34:10,638 - mmdet - INFO - Epoch [1][600/1061]	lr: 5.000e-04, eta: 8:19:56, time: 0.573, data_time: 0.011, memory: 18742, loss_rpn_cls: 0.0521, loss_rpn_bbox: 0.0176, loss_cls: 0.1022, acc: 96.9851, loss_bbox: 0.1109, loss: 0.2828
2022-02-11 12:36:04,006 - mmdet - INFO - Epoch [1][800/1061]	lr: 5.000e-04, eta: 8:16:55, time: 0.567, data_time: 0.010, memory: 18742, loss_rpn_cls: 0.0425, loss_rpn_bbox: 0.0150, loss_cls: 0.0937, acc: 97.1282, loss_bbox: 0.1106, loss: 0.2618
2022-02-11 12:37:56,

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 8561/8561, 13.2 task/s, elapsed: 650s, ETA:     0s

2022-02-11 12:49:24,365 - mmdet - INFO - Evaluating bbox...


...

2022-02-12 05:41:49,046 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.328
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.710
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.228
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.112
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.383
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.410
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.410
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.410
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.245
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.461
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100

DONE (t=0.36s).


2022-02-12 05:43:45,577 - mmdet - INFO - Epoch [50][200/1061]	lr: 5.000e-06, eta: 0:07:47, time: 0.582, data_time: 0.024, memory: 21208, loss_rpn_cls: 0.0040, loss_rpn_bbox: 0.0067, loss_cls: 0.0468, acc: 98.1609, loss_bbox: 0.1108, loss: 0.1683
2022-02-12 05:45:41,428 - mmdet - INFO - Epoch [50][400/1061]	lr: 5.000e-06, eta: 0:05:59, time: 0.579, data_time: 0.011, memory: 21208, loss_rpn_cls: 0.0055, loss_rpn_bbox: 0.0072, loss_cls: 0.0487, acc: 98.0967, loss_bbox: 0.1124, loss: 0.1738
2022-02-12 05:47:35,967 - mmdet - INFO - Epoch [50][600/1061]	lr: 5.000e-06, eta: 0:04:10, time: 0.573, data_time: 0.011, memory: 21208, loss_rpn_cls: 0.0048, loss_rpn_bbox: 0.0066, loss_cls: 0.0450, acc: 98.2424, loss_bbox: 0.1041, loss: 0.1605
2022-02-12 05:49:31,240 - mmdet - INFO - Epoch [50][800/1061]	lr: 5.000e-06, eta: 0:02:21, time: 0.576, data_time: 0.011, memory: 21208, loss_rpn_cls: 0.0050, loss_rpn_bbox: 0.0069, loss_cls: 0.0466, acc: 98.1670, loss_bbox: 0.1106, loss: 0.1691
2022-02-12 05:51

[>>>>>>>>>>>>>>>>>>>>>>>   ] 7878/8561, 13.2 task/s, elapsed: 595s, ETA:    52s

KeyboardInterrupt: 

In [22]:
# the new config inherits the base configs to highlight the necessary modification
_base_ = "/app/_data/mmdetection/configs/swin/mask_rcnn_swin-s-p4-w7_fpn_fp16_ms-crop-3x_coco.py"

In [23]:
_base_

'/app/_data/mmdetection/configs/swin/mask_rcnn_swin-s-p4-w7_fpn_fp16_ms-crop-3x_coco.py'

In [24]:
# 1. dataset settings
dataset_type = "CocoDataset"
classes = "fish"
data = dict(
    samples_per_gpu=2,
    workers_per_gpu=8,
    train=dict(
        type=dataset_type,
        # explicitly add your class names to the field `classes`
        classes=classes,
        ann_file="path/to/your/train/annotation_data",
        img_prefix="path/to/your/train/image_data",
    ),
    val=dict(
        type=dataset_type,
        # explicitly add your class names to the field `classes`
        classes=classes,
        ann_file="path/to/your/val/annotation_data",
        img_prefix="path/to/your/val/image_data",
    ),
    test=dict(
        type=dataset_type,
        # explicitly add your class names to the field `classes`
        classes=classes,
        ann_file="path/to/your/test/annotation_data",
        img_prefix="path/to/your/test/image_data",
    ),
)

# 2. model settings

# explicitly over-write all the `num_classes` field from default 80 to 5.
model = dict(
    roi_head=dict(
        bbox_head=[
            dict(
                type="Shared2FCBBoxHead",
                # explicitly over-write all the `num_classes` field from default 80 to 5.
                num_classes=1,
            ),
            dict(
                type="Shared2FCBBoxHead",
                # explicitly over-write all the `num_classes` field from default 80 to 5.
                num_classes=1,
            ),
            dict(
                type="Shared2FCBBoxHead",
                # explicitly over-write all the `num_classes` field from default 80 to 5.
                num_classes=1,
            ),
        ],
        # explicitly over-write all the `num_classes` field from default 80 to 5.
        mask_head=dict(num_classes=1),
    )
)